In [1]:
import glob
import pandas as pd
from scipy.io import loadmat
import numpy as np

In [2]:
col_list = ['Npoints', 
            'lapse1',
            'lapse2', 
            'deckchoice', 
            'RT1', 
            'RT2', 
            'choice_onset', 
            'press1_onset', 
            'info_onset', 
            'partner_onset', 
            'press2_onset', 
            'aff_onset', 
            'fix1_list', 
            'fix2_list', 
            'fix3_list', 
            'fix4_list', 
            'soc_win', 
            'block', 
            'duration', 
            'frequency', 
            'stimaff', 
            'stiminf', 
            'point_total', 
            'word', 
            'rating', 
            'is_catch', 
            'partner', 
            'highval_count']
file_list = glob.glob('data/10*/*[0-4].mat')

In [21]:
percent_switches = []
percent_stays = []

percent_stays_social_current = []
percent_switches_social_current = []

percent_stays_nonsocial_current = []
percent_switches_nonsocial_current = []

percent_stays_social_previous = []
percent_switches_social_previous = []

percent_stays_nonsocial_previous = []
percent_switches_nonsocial_previous = []

#information lower previous feedback
#info lower
percent_info_lower = []

#info higher 
percent_info_higher = []

#info lower previous -social 
percent_info_lower_sosical = []
#info higher previous -social
percent_info_higher_sosical = []

#info lower previous -nonsocial
percent_info_lower_nonsosical = []
#info higher previous -nonsocial
percent_info_higher_nonsosical = []

#--affective feedback
#win prior 
percent_win_switches = []
percent_win_stays = []
#loss prior
percent_loss_switches = []
percent_loss_stays = []

#win prior- social
percent_win_switches_social = []
percent_win_stays_social = []

#loss prior- social
percent_loss_switches_social = []
percent_loss_stays_social = []

#win prior- nonsocial
percent_win_switches_nonsocial = []
percent_win_stays_nonsocial = []

#loss prior- nonsocial
percent_loss_switches_nonsocial = []
percent_loss_stays_nonsocial = []


for file in file_list:
    df = pd.DataFrame([[row.flat[0] for row in line] for line in loadmat(file)['data'][0]], columns=col_list)
    #df.to_csv(file[10:-4] + ".csv", index = False)
    
    df = pd.DataFrame([[row.flat[0] for row in line] for line in loadmat(file)['data'][0]], columns=col_list)
    
    li = list(df.deckchoice)
    
    stay_switch = []
    for i, row in enumerate(li):
        if i > 0:
            if row == li[i-1]:
                stay_switch.append('stay')
            elif (row == 11 and li[i-1] == 1) or (row == 22 and li[i-1] == 2):
                stay_switch.append('stay-noaff')
            elif (row == 1 and li[i-1] == 11) or (row == 2 and li[i-1] == 22):
                stay_switch.append('stay')    
            elif (row == 11 and li[i-1] != 1) or (row == 22 and li[i-1] != 2):
                stay_switch.append('switch-noaff')
            else:
                stay_switch.append('switch')
        else:
            stay_switch.append('n/a')
                
    df['stay_switch'] = stay_switch
        
    #current social partner column == 1 is social

    
    #previous social prev_partner == 1 is social
    li = list(df.partner)
    
    prev_partner = []
    for i, row in enumerate(li):
        if i > 0:
            if li[i-1] == 1:
                prev_partner.append(1)
            elif li[i-1] == 0:
                prev_partner.append(0)
        else:
            prev_partner.append('n/a')
                
    df['prev_partner'] = prev_partner
    
    #info current greater or less than current == 1 is greater, 0 is less, 2 is same
    
    li = list(df.Npoints)
    
    rel_points = []
    for i, row in enumerate(li):
        if i > 0:
            if li[i-1] > li[i]:
                rel_points.append(0)
            elif li[i-1] < li[i]:
                rel_points.append(1)
            elif li[i-1] == li[i]:
                rel_points.append(2)
                
        else:
            rel_points.append('n/a')
    
    
    
    df['rel_points'] = rel_points
    
    #win loss prior == 1 is win
    
    li = list(df.soc_win)   
    
    prior_win_loss = []
    
    for i, row in enumerate(li):
        if i > 0:
            if li[i-1] == 1:
                prior_win_loss.append(1)
            elif li[i-1] == 0:
                prior_win_loss.append(0)
        else:
            prior_win_loss.append('n/a')
    
    df['prior_win_loss'] = prior_win_loss
    
    #stay/switch
    valid_stays = df.apply(lambda x: True if x['lapse1'] == 0 and x['lapse2'] == 0 and x['stay_switch'] =='stay' else False , axis=1)
    valid_switches = df.apply(lambda x: True if x['lapse1'] == 0 and x['lapse2'] == 0 and x['stay_switch'] =='switch' else False , axis=1)
    
    count_valid_stays = len(valid_stays[valid_stays == True].index)
    count_valid_switches = len(valid_stays[valid_switches == True].index)
    count_valid_stay_switch = count_valid_stays + count_valid_switches
    
    percent_stays.append(count_valid_stays / count_valid_stay_switch)
    percent_switches.append(count_valid_switches / count_valid_stay_switch)
    
    #social current
    valid_stays_social_current = df.apply(lambda x: True if x['lapse1'] == 0 and x['lapse2'] == 0 and x['stay_switch'] =='stay' and x['partner'] == 1 else False , axis=1)
    valid_switches_social_current = df.apply(lambda x: True if x['lapse1'] == 0 and x['lapse2'] == 0 and x['stay_switch'] =='switch' and x['partner'] == 1 else False , axis=1)
    
    count_valid_stays_social_current = len(valid_stays_social_current[valid_stays_social_current == True].index)
    count_valid_switches_social_current = len(valid_switches_social_current[valid_switches_social_current == True].index)
    count_valid_stay_switch_social_current = count_valid_stays_social_current + count_valid_switches_social_current
    
    percent_stays_social_current.append(count_valid_stays_social_current / count_valid_stay_switch_social_current)
    percent_switches_social_current.append(count_valid_switches_social_current / count_valid_stay_switch_social_current)
    
    #nonsocial current
    valid_stays_nonsocial_current = df.apply(lambda x: True if x['lapse1'] == 0 and x['lapse2'] == 0 and x['stay_switch'] =='stay' and x['partner'] == 0 else False , axis=1)
    valid_switches_nonsocial_current = df.apply(lambda x: True if x['lapse1'] == 0 and x['lapse2'] == 0 and x['stay_switch'] =='switch' and x['partner'] == 0 else False , axis=1) 
    
    count_valid_stays_nonsocial_current = len(valid_stays_nonsocial_current[valid_stays_nonsocial_current == True].index)
    count_valid_switches_nonsocial_current = len(valid_switches_nonsocial_current[valid_switches_nonsocial_current == True].index)
    count_valid_stay_switch_nonsocial_current = count_valid_stays_nonsocial_current + count_valid_switches_nonsocial_current
    
    percent_stays_nonsocial_current.append(count_valid_stays_nonsocial_current / count_valid_stay_switch_nonsocial_current)
    percent_switches_nonsocial_current.append(count_valid_switches_nonsocial_current / count_valid_stay_switch_nonsocial_current) 
    
    #social previous
    valid_stays_social_previous= df.apply(lambda x: True if x['lapse1'] == 0 and x['lapse2'] == 0 and x['stay_switch'] =='stay' and x['prev_partner'] == 1 else False , axis=1)
    valid_switches_social_previous = df.apply(lambda x: True if x['lapse1'] == 0 and x['lapse2'] == 0 and x['stay_switch'] =='switch' and x['prev_partner'] == 1 else False , axis=1)
    
    count_valid_stays_social_previous = len(valid_stays_social_previous[valid_stays_social_previous == True].index)
    count_valid_switches_social_previous = len(valid_switches_social_previous[valid_switches_social_previous == True].index)
    count_valid_stay_switch_social_previous = count_valid_stays_social_previous + count_valid_switches_social_previous
    
    percent_stays_social_previous.append(count_valid_stays_social_previous / count_valid_stay_switch_social_previous)
    percent_switches_social_previous.append(count_valid_switches_social_previous / count_valid_stay_switch_social_previous)
    
    #nonsocial previous
    valid_stays_nonsocial_previous = df.apply(lambda x: True if x['lapse1'] == 0 and x['lapse2'] == 0 and x['stay_switch'] =='stay' and x['prev_partner'] == 0 else False , axis=1)
    valid_switches_nonsocial_previous = df.apply(lambda x: True if x['lapse1'] == 0 and x['lapse2'] == 0 and x['stay_switch'] =='switch' and x['prev_partner'] == 0 else False , axis=1)
    
    count_valid_stays_nonsocial_previous = len(valid_stays_nonsocial_previous[valid_stays_nonsocial_previous == True].index)
    count_valid_switches_nonsocial_previous = len(valid_switches_nonsocial_previous[valid_switches_nonsocial_previous == True].index)
    count_valid_stay_switch_nonsocial_previous = count_valid_stays_nonsocial_previous + count_valid_switches_nonsocial_previous
    
    percent_stays_nonsocial_previous.append(count_valid_stays_nonsocial_previous / count_valid_stay_switch_nonsocial_previous)
    percent_switches_nonsocial_previous.append(count_valid_switches_nonsocial_previous / count_valid_stay_switch_nonsocial_previous)
    
    #info lower/higher
    valid_info_lower_stays_previous = df.apply(lambda x: True if x['lapse1'] == 0 and x['lapse2'] == 0 and x['rel_points'] =='stay' else False , axis=1)
    valid_info_lower_switches_previous = df.apply(lambda x: True if x['lapse1'] == 0 and x['lapse2'] == 0 and x['rel_points'] =='switch' else False , axis=1)
    
    count_valid_info_lower_stays_previous = len(valid_info_lower_stays_previous[valid_info_lower_stays_previous == True].index)
    count_valid_info_lower_switches_previous = len(valid_info_lower_switches_previous[valid_info_lower_switches_previous == True].index)
    count_valid_stay_switch_info_lower_previous = count_valid_stays_nonsocial_previous + count_valid_switches_nonsocial_previous
    
    percent_info_lower.append(count_valid_info_lower_stays_previous / count_valid_stay_switch_info_lower_previous)
    percent_info_higher.append(count_valid_info_lower_switches_previous / count_valid_stay_switch_info_lower_previous)
    
    
    #info lower previous -social 
    #info higher previous -social
    
    
    percent_info_lower_sosical = []
    percent_info_higher_sosical = []
    
    
    
    
    #info lower previous -nonsocial 
    
    #info higher previous -nonsocial
    ##################################################
    
    #win prior 
    valid_win_stays_previous = df.apply(lambda x: True if x['lapse1'] == 0 and x['lapse2'] == 0 and x['stay_switch'] =='stay' and x['prior_win_loss'] == 1 else False , axis=1)
    valid_win_switches_previous = df.apply(lambda x: True if x['lapse1'] == 0 and x['lapse2'] == 0 and x['stay_switch'] =='switch' and x['prior_win_loss'] == 1 else False , axis=1)
    
    count_valid_win_stays_previous = len(valid_win_stays_previous[valid_win_stays_previous == True].index)
    count_valid_win_switches_previous = len(valid_win_switches_previous[valid_win_switches_previous == True].index)
    count_valid_win_stay_switches_previous = count_valid_stays_nonsocial_previous + count_valid_switches_nonsocial_previous
    
    percent_win_switches.append(count_valid_win_switches_previous / count_valid_win_stay_switches_previous)
    percent_win_stays.append(count_valid_win_stays_previous  / count_valid_win_stay_switches_previous)
    
    
    
    #loss prior
    valid_loss_stays_previous = df.apply(lambda x: True if x['lapse1'] == 0 and x['lapse2'] == 0 and x['stay_switch'] =='stay' and x['prior_win_loss'] == 0 else False , axis=1)
    valid_loss_switches_previous = df.apply(lambda x: True if x['lapse1'] == 0 and x['lapse2'] == 0 and x['stay_switch'] =='switch' and x['prior_win_loss'] == 0 else False , axis=1)
    
    count_valid_loss_stays_previous = len(valid_loss_stays_previous[valid_loss_stays_previous == True].index)
    count_valid_loss_switches_previous = len(valid_loss_switches_previous[valid_loss_switches_previous == True].index)
    count_valid_loss_stay_switches_previous = count_valid_loss_stays_previous + count_valid_loss_switches_previous
    
    percent_loss_switches.append(count_valid_loss_stays_previous / count_valid_loss_stay_switches_previous)
    percent_loss_stays.append(count_valid_loss_switches_previous / count_valid_loss_stay_switches_previous)
    
    #print(percent_loss_switches, percent_win_stays)
    
    #########################################
    #win prior- social
    
    #loss prior- social
    
    #win prior- nonsocial
    
    #loss prior- nonsocial
    
print(percent_loss_switches, percent_win_stays)

[0.46153846153846156, 0.25, 0.35714285714285715, 0.4444444444444444, 0.4444444444444444, 0.5555555555555556, 0.5, 0.4666666666666667, 0.4444444444444444, 0.75, 0.75, 0.631578947368421, 0.5294117647058824, 0.5789473684210527, 0.5882352941176471, 0.65, 0.625, 0.9411764705882353, 0.8125, 0.8125, 0.7894736842105263, 0.7692307692307693, 0.7777777777777778, 0.8947368421052632, 1.0, 0.8823529411764706, 0.8823529411764706, 0.7647058823529411, 0.9473684210526315, 0.7777777777777778, 0.75, 0.7058823529411765, 0.6842105263157895, 0.6111111111111112, 0.5, 0.35294117647058826, 0.14285714285714285, 0.3888888888888889, 0.2, 0.5555555555555556, 0.2777777777777778, 0.29411764705882354, 0.4375, 0.21428571428571427, 0.3888888888888889, 0.5789473684210527, 0.35294117647058826, 0.16666666666666666, 0.5625, 0.5, 0.6470588235294118, 0.47058823529411764, 0.5555555555555556, 0.6666666666666666, 0.7222222222222222, 0.631578947368421, 0.7368421052631579, 0.5333333333333333, 0.75, 0.3888888888888889, 0.5555555555

In [146]:
#social vs non social
#points


#spreadsheet for stay|switch
#per subject | social previous| non social previous | social current| non-social current | info better than previous |info worse than previous|info is same |win prior | loss prior 

#% over valid trials - lapses and - catch

#confirm onset 0 points across 

#make stay_switch column
    #df['stay_switch'] =  
    #choice_lapse = df[df.lapse1 == 1]
    #choice_lapse['duration'] = '3000'
    #choice_lapse['trial_type'] = 'choice_nonsoc'
    #choice_lapse = choice_lapse[['choice_onset', 'duration', 'trial_type']]
    #print(choice_lapse)
    #choice_lapse = choice_lapse.rename({'choice_onset': 'onset'}, axis=1)
    
    #choice_lapse1 = df[df.lapse1 == 2]
    #choice_lapse1['duration'] = '3000'
    #choice_lapse1['trial_type'] = 'choice_nonsoc'
    #choice_lapse1 = choice_lapse[['choice_onset', 'duration', 'trial_type']]
    #choice_lapse1 = choice_lapse.rename({'choice_onset': 'onset'}, axis=1)
    
    #outcome_aff_w_soc = df[df.partner == 1][df.soc_win == 1][df.lapse1 == 0][df.lapse2 == 0]
    #outcome_aff_w_soc['duration'] = '1500' 
    #outcome_aff_w_soc['trial_type'] = 'outcome_aff_w_soc'
    #outcome_aff_w_soc = outcome_aff_w_soc[['aff_onset', 'duration', 'trial_type']]
    #outcome_aff_w_soc = outcome_aff_w_soc.rename({'aff_onset': 'onset'}, axis=1)
    
    #outcome_aff_w_nonsoc = df[df.partner == 0][df.soc_win == 1][df.lapse1 == 0][df.lapse2 == 0]
    #outcome_aff_w_nonsoc['duration'] = '1500' 
    #outcome_aff_w_nonsoc['trial_type'] = 'outcome_aff_w_nonsoc'
    #outcome_aff_w_nonsoc = outcome_aff_w_nonsoc[['aff_onset', 'duration', 'trial_type']]
    #outcome_aff_w_nonsoc = outcome_aff_w_nonsoc.rename({'aff_onset': 'onset'}, axis=1)
    
    #outcome_aff_l_soc = df[df.partner == 1][df.soc_win == 0][df.lapse1 == 0][df.lapse2 == 0]
    #outcome_aff_l_soc['duration'] = '1500' 
    #outcome_aff_l_soc['trial_type'] = 'outcome_aff_l_soc'
    #outcome_aff_l_soc = outcome_aff_l_soc[['aff_onset', 'duration', 'trial_type']]
    #outcome_aff_l_soc = outcome_aff_l_soc.rename({'aff_onset': 'onset'}, axis=1)
    
    #outcome_aff_l_nonsoc = df[df.partner == 0][df.soc_win == 0][df.lapse1 == 0][df.lapse2 == 0]
    #outcome_aff_l_nonsoc['duration'] = '1500' 
    #outcome_aff_l_nonsoc['trial_type'] = 'outcome_aff_l_nonsoc'
    #outcome_aff_l_nonsoc = outcome_aff_l_nonsoc[['aff_onset', 'duration', 'trial_type']]
    #outcome_aff_l_nonsoc = outcome_aff_l_nonsoc.rename({'aff_onset': 'onset'}, axis=1)
    
    #outcome_info_soc = df[df.partner == 1][df.lapse1 == 0][df.lapse2 == 0]
    #outcome_info_soc['duration'] = '1500'
    #outcome_info_soc['trial_type'] = 'outcome_info_soc'
    #outcome_info_soc = outcome_info_soc[['info_onset', 'duration', 'trial_type']]
    #outcome_info_soc = outcome_info_soc.rename({'info_onset': 'onset'}, axis=1)
    
    #outcome_info_nonsoc = df[df.partner == 0][df.lapse1 == 0][df.lapse2 == 0]
    #outcome_info_nonsoc['duration'] = '1500'
    #outcome_info_nonsoc['trial_type'] = 'outcome_info_nonsoc'
    #outcome_info_nonsoc = outcome_info_nonsoc[['info_onset', 'duration', 'trial_type']]
    #outcome_info_nonsoc = outcome_info_nonsoc.rename({'info_onset': 'onset'}, axis=1)
    
    #choice_soc = df[df.partner == 1][df.lapse1 == 0][df.lapse2 == 0]
    #choice_soc['duration'] = '1500'
    #choice_soc['trial_type'] = 'choice_soc'
    #choice_soc = choice_soc[['choice_onset', 'duration', 'trial_type']]
    #choice_soc = choice_soc.rename({'choice_onset': 'onset'}, axis=1)
    
    #choice_nonsoc = df[df.partner == 0][df.lapse1 == 0][df.lapse2 == 0]
    #choice_nonsoc['duration'] = '3000'
    #choice_nonsoc['trial_type'] = 'choice_nonsoc'
    #choice_nonsoc = choice_nonsoc[['choice_onset', 'duration', 'trial_type']]
    #choice_nonsoc = choice_nonsoc.rename({'choice_onset': 'onset'}, axis=1)
    
    #merged_df = pd.concat([
    #                       choice_lapse,
                           #choice_lapse1
        
    #                       ])
    
    #merged_df = merged_df[merged_df.onset != 0.00]

105
